In [ ]:
test_continuous_features = list(X_test.select_dtypes(include=['int']).columns)
test_categorical_features = list(X_test.select_dtypes(include=['object']).columns)

for col in test_continuous_features:
    X_test[col] = X_test[col].astype('int32')
for col in test_categorical_features:
    X_test[col] = X_test[col].astype('category')

test_continuous = np.stack([X_test[col].values for col in test_continuous_features], 1)
test_categorical = np.stack([X_test[col].cat.codes.values for col in test_categorical_features], 1)

y = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)
test_continuous = torch.tensor(test_continuous, dtype=torch.float32)
test_categorical = torch.tensor(test_categorical, dtype=torch.int32)

test_category_sizes = [len(X_test[col].cat.categories) for col in test_categorical_features]
test_embedding_sizes = [(size, min(50, (size+1)//2)) for size in test_category_sizes]
test_self_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in test_embedding_sizes])

embeddings = []
for i,e in enumerate(test_self_embeddings):
    embeddings.append(e(test_categorical[:,i]))

embedded_categorical = torch.cat(embeddings, 1)
X_test_tensor = torch.cat((test_continuous, embedded_categorical), 1)
test_datset = TensorDataset(X_test_tensor, embedded_categorical)
test_dataloader = DataLoader(test_datset, batch_size=20, shuffle=True)